## Read Files 

### hw3dataset
Each row represents a directed edge (link) between nodes separated by a comma.
The direction of a edge is from the first node to the second node.
```
    graph_1.txt: 6 nodes, 5 edges
    graph_2.txt: 5 nodes, 5 edges (a circle)
    graph_3.txt: 4 nodes, 6 edges
    graph_4.txt: 7 nodes, 18 edges (the example in Lecture3, p29)
    graph_5.txt: 469 nodes, 1102 edges
    graph_6.txt: 1228 nodes, 5220 edges
```
### Transaction Dataset (in hw1)

In [182]:

import os
from collections import defaultdict 

import numpy as np 

from easydict import EasyDict as edict
from utils import timer 
# No self loops 

filedir ='./data/'
edges_data = {} 
for filename in os.listdir(filedir):
    edges = [] 
    filepath = os.path.join(filedir, filename) 
    print(f'reading {filepath}...')
    filesuff = filename.split('.')[0] 
    if filename.startswith('graph'):
        with open(filepath, 'r') as f: 
            for line in f.readlines():
                line = line.strip()
                edge = line.split(',')
                edges.append(edge)  
        edges_data[filesuff] = edges
                
    elif filename.startswith('ibm'):
        with open(filepath, 'r') as f: 
            for line in f.readlines():
                line = line.strip()
                edge = line.split()[1:]
                edges.append(edge)
        edges_data[filesuff] = edges
    
    
    
                    

class Graph:
    def __init__(self, edges):
        self.out_neighbors = defaultdict(list)
        self.in_neighbors = defaultdict(list)
        nodes = set()
        for u, v in edges:
            nodes.add(u); nodes.add(v) 
        nodes = sorted(nodes)
        print(nodes[:10])    
        nodesmap = {node:nodeidx for nodeidx, node in enumerate(nodes)}
        for u, v in edges:
            u, v = nodesmap[u], nodesmap[v]
            self.out_neighbors[u].append(v)
            self.in_neighbors[v].append(u)  
        self.N = len(nodes)
edges_data = sorted(edges_data.items(), key = lambda x:x[0])

reading ./data/graph_4.txt...
reading ./data/graph_5.txt...
reading ./data/graph_6.txt...
reading ./data/graph_2.txt...
reading ./data/.DS_Store...
reading ./data/graph_3.txt...
reading ./data/graph_1.txt...
reading ./data/ibm-5000.txt...


In [161]:
from pprint import pprint
Graphs = {}
for fname, edges in edges_data:
    G = Graph(edges)
    Graphs[fname] = G
    print(fname, f': graph with {G.N} nodes')
    # pprint(G.out_neighbors)

['1', '2', '3', '4', '5', '6']
graph_1 : graph with 6 nodes
['1', '2', '3', '4', '5']
graph_2 : graph with 5 nodes
['1', '2', '3', '4']
graph_3 : graph with 4 nodes
['1', '2', '3', '4', '5', '6', '7']
graph_4 : graph with 7 nodes
['1', '10', '100', '101', '102', '103', '104', '105', '106', '107']
graph_5 : graph with 469 nodes
['1', '10', '100', '1000', '1001', '1002', '1003', '1004', '1005', '1006']
graph_6 : graph with 1228 nodes
['1', '10', '100', '101', '102', '103', '104', '105', '106', '107']
ibm-5000 : graph with 836 nodes


In [183]:

def nodeid(node:str):
    return int(node)-1 

def PageRank(G:Graph, 
            max_iters:int, 
            damping_factor:float):
    """
    Args:
        G (networkx.classes.graph.Graph): 
        max_iters (int): number of iters 
        damping_factor (float): 
        The PageRank theory holds that an imaginary surfer who is randomly clicking on links will eventually stop clicking. The probability, at any step, that the person will continue is a damping factor d.
    Note that index 0 is null
    Note that links from a page to itself are ignored 
    """
    N = G.N
    if N == 0:
        raise ValueError('Empty Graph')
    PageRanksHistory = []  
    dpfactor = damping_factor
    # init 
    PageRanks = np.full(N, 1/N)
    for iter in range(max_iters):
        newPageRanks = np.zeros_like(PageRanks)
        for i in range(N):
            # for every node points to me , update page_rank score as sum of (old_page[ni]) / (ni out links)
            for n in G.in_neighbors[i]:
                newPageRanks[i] += PageRanks[n] / len(G.out_neighbors[n])
        PageRanks = (1-dpfactor) * newPageRanks + dpfactor/N
    PageRanks = PageRanks / (PageRanks.sum())
    return PageRanks

In [ ]:
for filename, g in Graphs.items():
    print(filename)
    pageranks = PageRank(g, max_iters=100, damping_factor = 0.15)
    print(pageranks)
    

In [184]:


from typing import Tuple
def HITS(G:Graph, 
    max_iters:int, 
    denominator = 'sum')-> Tuple[np.array, np.array]:
    """
    HITS(Hyperlink-induced topic search)
    Authority: Providing valuable infor on certain topic 
    Hub: Give good supports to those pages with high authority
    - A good hub increases the authority weight of the pages it points. 
    - A good authority increases the hub weight of the pages that point to it. 
    The idea is then to apply the two operations above alternatively until equilibrium values for the hub and authority weights are reached.
    Args:
        G (Graph): _description_
    Returns:
        Tuple(np.array, np.array): Auth, Hub Vectors 
            Auth: shape (N, ) Auth[n] is the authority score of node n
            Hub: shape (N, )  Similarly, Hub[n] is the hub score of node n
    """
    auths = np.ones(G.N)
    hubs = np.ones(G.N)
    
    def get_update_Auth(n):
        # authority: the node being pointed to 越多人指向他越高分
        return hubs[G.in_neighbors[n]].sum()
    def get_update_Hub(n):
        return auths[G.out_neighbors[n]].sum()
    
    for _ in range(max_iters):
        new_auths = np.zeros_like(auths)
        new_hubs = np.zeros_like(hubs)
        for n in range(G.N):
            new_auths[n] = get_update_Auth(n)
            new_hubs[n] = get_update_Hub(n)
        if denominator == 'sum':
            auths = new_auths / np.sum(new_auths)
            hubs = new_hubs / np.sum(new_hubs)
        else: # root of sum of squares
            # 這是wik上上面的正規做法
            # https://en.wikipedia.org/wiki/HITS_algorithm
            auths = new_auths / np.sqrt(np.sum(new_auths**2))
            hubs = new_hubs / np.sqrt(np.sum(new_hubs**2))
    
    return auths, hubs 


In [ ]:
for filename, g in Graphs.items():
    print(filename)
    auths, hubs = HITS(g, max_iters=100)
    print(list(auths))
    print(list(hubs))

In [185]:
@timer 
def SimRank(G: Graph, 
            max_iters:int, 
            decay_factor:float):
    # SimRank_sum = the sum of SimRank value of all in-neighbor pairs (SimRank value is from the previous iteration)
    C = decay_factor 
    def get_update_simrank(a:int, b:int, simRank: np.array):
        if a == b: 
            return 1    
        a_in_neighbors = G.in_neighbors[a]
        b_in_neighbors = G.in_neighbors[b]
        a_in_size, b_in_size = len(a_in_neighbors), len(b_in_neighbors)
        if not (a_in_size and b_in_size):
            return 0
        temp = 0 
        for ai in a_in_neighbors:
            for bi in b_in_neighbors:
                temp += simRank[ai, bi]
        # scaling the simRank 
        return C * temp / (a_in_size * b_in_size) 
                        

    simRank = np.zeros((G.N, G.N))
    for iter in range(max_iters):
        newSimRank = np.zeros_like(simRank)
        for a in range(G.N):
            for b in range(a, G.N):
                newSimRank[a, b] = newSimRank[b, a] = get_update_simrank(a, b, simRank)
        simRank = newSimRank 
    return simRank    

In [187]:
for filename, g in Graphs.items():
    print(filename)
    simranks = SimRank(g, max_iters=100, decay_factor = 0.9)
    print(simranks)

graph_1
SimRank Done in 0.01 seconds.
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
graph_2
SimRank Done in 0.01 seconds.
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
graph_3
SimRank Done in 0.01 seconds.
[[1.         0.         0.81818182 0.        ]
 [0.         1.         0.         0.81818182]
 [0.81818182 0.         1.         0.        ]
 [0.         0.81818182 0.         1.        ]]
graph_4
SimRank Done in 0.01 seconds.
[[1.         0.56308319 0.55339919 0.55602199 0.54405249 0.6027948
  0.50924918]
 [0.56308319 1.         0.59679481 0.56752335 0.60393344 0.50222034
  0.63282635]
 [0.55339919 0.59679481 1.         0.62843716 0.58456124 0.62685197
  0.63002234]
 [0.55602199 0.56752335 0.62843716 1.         0.54545408 0.69482362
  0.69482362]
 [0.54405249 0.60393344 0.58456124 0.54545408 1.         0.49059331
  0.60031484]
 [0.6027948  0.50222034 0.626851